In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA default;

In [0]:
%sql
DROP TABLE IF EXISTS default.courses;
DROP TABLE IF EXISTS default.courses_autoloader_sink;


In [0]:
test_file_to_delete = "/Volumes/workspace/default/volume/courses_part2.json"

# --- 1. Tabellen löschen, falls sie existieren ---
spark.sql("DROP TABLE IF EXISTS workspace.default.courses_autoloader_sink")
print("✅ Sink-Tabellen gelöscht (falls vorhanden).")

# --- 2. Das zweite JSON-Testfile löschen, falls es existiert ---
try:
  dbutils.fs.rm(test_file_to_delete)
  print(f"✅ Test-Datei '{test_file_to_delete}' wurde gelöscht.")
except Exception as e:
  if 'FileNotFoundException' in str(e):
    print(f"Test-Datei '{test_file_to_delete}' existierte nicht, keine Aktion erforderlich.")
  else:
    raise e # Andere Fehler trotzdem anzeigen

# --- 3. Alle Checkpoints löschen ---
try:
  dbutils.fs.rm(autoloader_checkpoint, recurse=True)
  print(f"✅ Auto Loader Checkpoint gelöscht.")
except:
  print("Auto Loader Checkpoint existierte nicht, keine Aktion erforderlich.")

print("\n🚀 System ist vollständig zurückgesetzt und bereit für einen neuen Durchlauf.")

✅ Sink-Tabellen gelöscht (falls vorhanden).
✅ Test-Datei '/Volumes/workspace/default/volume/courses_part2.json' wurde gelöscht.
Auto Loader Checkpoint existierte nicht, keine Aktion erforderlich.

🚀 System ist vollständig zurückgesetzt und bereit für einen neuen Durchlauf.


In [0]:
dbutils.fs.rm("/Volumes/workspace/default/volume/_chk/autoloader_courses", recurse=True)

False

# Vorbereitung: Datei einlesen und als Tabelle abspeichern

In [0]:
%python
dataset_school = "/Volumes/workspace/default/volume"

all_files = dbutils.fs.ls(dataset_school)
json_files = [f for f in all_files if f.name.endswith(".json")]

display(json_files)

path,name,size,modificationTime
dbfs:/Volumes/workspace/default/volume/courses.json,courses.json,329,1756365194000
dbfs:/Volumes/workspace/default/volume/enrollments.json,enrollments.json,371,1756365193000
dbfs:/Volumes/workspace/default/volume/nested_json.json,nested_json.json,1186,1756365194000
dbfs:/Volumes/workspace/default/volume/students.json,students.json,322,1756365194000
dbfs:/Volumes/workspace/default/volume/students_update.json,students_update.json,216,1756365193000


# Auto Loader

In [0]:
# Pfad zum Ordner, den der Auto Loader überwachen soll
source_path = "/Volumes/workspace/default/volume/"

# Pfad für den Checkpoint, damit sich der Auto Loader merkt, was er gelesen hat
checkpoint_path = "/Volumes/workspace/default/volume/_chk/autoloader_courses"

# Ziel-Tabelle
target_table = "courses_autoloader_sink"

In [0]:

# Auto Loader Stream definieren und starten
autoloader_stream = (
  spark.readStream
    .format("cloudFiles")  # Auto Loader aktivieren
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", checkpoint_path) # Schema-Infos hier speichern
    .option("pathGlobFilter", "courses*.json") # Nur Dateien verarbeiten, die mit "courses" beginnen
    .load(source_path)
.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path)
    .option("mergeSchema", "true") # Erlaubt das Hinzufügen neuer Spalten
    .trigger(availableNow=True) # Notwendig für die Free Edition
    .toTable(target_table)
)

autoloader_stream.awaitTermination() # blockiert die Ausführung der Notebook-Zelle und wartet, bis der Streaming-Job beendet ist.

print(f"✅ Auto Loader hat die Daten erfolgreich in die Tabelle '{target_table}' geladen.")

✅ Auto Loader hat die Daten erfolgreich in die Tabelle 'courses_autoloader_sink' geladen.


In [0]:
%sql
SELECT * FROM workspace.default.courses_autoloader_sink;

category,course_id,instructor,price,title,_rescued_data
Technology,101,Dr. Smith,850,Data Engineering,null
AI,102,Dr. Lee,1200,Machine Learning,null
Philosophy,103,Dr. Kim,600,Ethics in AI,null


In [0]:
# Inhalt für unsere neue Datei
json_data = """
[
  {"course_id": 301, "title": "Advanced SQL", "instructor": "Dr. Codd", "category": "Technology", "price": 300},
  {"course_id": 302, "title": "Data Warehousing", "instructor": "Dr. Codd", "category": "Technology", "price": 400},
  {"course_id": 104, "title": "Applied Ethics", "instructor": "Dr. Kim", "category": "Philosophy", "price": 250}
]
"""

# Pfad und Dateiname
file_path = "/Volumes/workspace/default/volume/courses_part2.json"

# Die neue Datei erstellen
dbutils.fs.put(file_path, json_data, overwrite=True)

print(f"✅ Neue Datei '{file_path}' wurde erfolgreich erstellt.")

Wrote 348 bytes.
✅ Neue Datei '/Volumes/workspace/default/volume/courses_part2.json' wurde erfolgreich erstellt.


In [0]:
%sql
-- 1. Führe die Auto-Loader-Zelle (Code-Block 9) erneut aus.
--    Der Stream wird nur die neue Datei "courses_part2.json" verarbeiten.

-- 2. Überprüfe das Endergebnis.
--    Die Tabelle enthält jetzt die Daten aus beiden Dateien.
SELECT * FROM workspace.default.courses_autoloader_sink ORDER BY course_id;

category,course_id,instructor,price,title,_rescued_data
Technology,101,Dr. Smith,850,Data Engineering,null
AI,102,Dr. Lee,1200,Machine Learning,null
Philosophy,103,Dr. Kim,600,Ethics in AI,null
Philosophy,104,Dr. Kim,250,Applied Ethics,null
Technology,301,Dr. Codd,300,Advanced SQL,null
Technology,302,Dr. Codd,400,Data Warehousing,null


In [0]:
%sql
DESCRIBE HISTORY courses_autoloader_sink

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-08-28T11:54:21.000Z,75300490606458,philippe.christen@students.fhnw.ch,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 1d3c865d-b41a-45fc-b3ad-4dbbae5078d1, epochId -> 1, statsOnLoad -> true)",null,List(1341295095209958),0828-112228-4mlwmr12-v2n,1,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 3, numOutputBytes -> 1788, numAddedFiles -> 1)",null,Databricks-Runtime/17.1.x-photon-scala2.13
1,2025-08-28T11:53:36.000Z,75300490606458,philippe.christen@students.fhnw.ch,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 1d3c865d-b41a-45fc-b3ad-4dbbae5078d1, epochId -> 0, statsOnLoad -> true)",null,List(1341295095209958),0828-112228-4mlwmr12-v2n,0,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 3, numOutputBytes -> 1746, numAddedFiles -> 1)",null,Databricks-Runtime/17.1.x-photon-scala2.13
0,2025-08-28T11:53:31.000Z,75300490606458,philippe.christen@students.fhnw.ch,CREATE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)",null,List(1341295095209958),0828-112228-4mlwmr12-v2n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/17.1.x-photon-scala2.13


 #Aufräumen

In [0]:
%sql
DROP TABLE IF EXISTS default.courses_autoloader_sink;